# Notebook 1 – Exploración de Datos (EDA) con Datos de Accidentes de Tránsito

**Objetivo del notebook:**  
Aprender a realizar una exploración de datos (EDA) básica y limpieza inicial usando un dataset real sobre accidentes de tránsito y seguros en distintos estados de EE. UU.

**Dataset:**  
Cada fila representa un estado y contiene indicadores de choques fatales y costos de seguros:

- `Estado`
- `Número de conductores involucrados en colisiones fatales por cada mil millones de millas`
- `Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad`
- `Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol`
- `Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos`
- `Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos`
- `Primas de seguro de automóvil ($)`
- `Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($)`

En este notebook vamos a:

1. Importar el dataset y entender su estructura.
2. Revisar tipos de datos, valores faltantes y duplicados.
3. Detectar outliers numéricos de forma simple.
4. Crear algunas variables nuevas que tengan sentido analítico.
5. Dejar un **dataframe limpio y listo** para usar en modelos en el siguiente notebook.



In [ ]:
# ============================================================
# 1. Importar librerías y cargar el dataset
# ============================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Opciones de visualización
pd.set_option('display.max_columns', None)
plt.style.use('default')

# Cargar el dataset (ajusta la ruta al archivo si es necesario)
# Por ejemplo: 'bad-drivers.csv' en la misma carpeta del notebook
df = pd.read_csv("bad-drivers.csv")

# Mirar las primeras filas
df.head()

## 2. Inspección inicial del dataset

En esta sección queremos responder rápidamente:

- ¿Cuántas filas y columnas tiene el dataset?
- ¿Qué tipo de dato tiene cada columna?
- ¿Hay valores faltantes?
- ¿Cómo lucen las estadísticas básicas de las columnas numéricas?



In [ ]:
# Dimensiones del dataset
print("Dimensiones del dataset (filas, columnas):", df.shape)

# Información general de tipos de datos y valores no nulos
df.info()

In [ ]:
# Estadísticas descriptivas de las columnas numéricas
df.describe().T

### Preguntas rápidas (para discutir en voz alta / comentar en celdas Markdown):

1. ¿Te llama la atención algún rango de valores (mínimo, máximo, promedio) en las variables numéricas?
2. ¿Hay columnas con valores muy diferentes entre el mínimo y el máximo que podrían indicar presencia de **outliers**?
3. ¿Qué columnas parecen claramente numéricas y cuál es la única claramente categórica?


## 3. Renombrar columnas a un formato más manejable

Los nombres originales son largos. Vamos a renombrarlos a `snake_case` para facilitar el trabajo en Python.

Además, esto es una **buena práctica** para futuros modelos.


In [ ]:
# Mostrar nombres de columnas originales
df.columns.tolist()

In [ ]:
# Diccionario de renombre de columnas

rename_dict = {
    "Estado": "estado",
    "Número de conductores involucrados en colisiones fatales por cada mil millones de millas": "conductores_fatales_mm_millas",
    "Porcentaje de conductores involucrados en colisiones fatales con exceso de velocidad": "pct_exceso_velocidad",
    "Porcentaje de conductores involucrados en colisiones fatales bajo la influencia del alcohol": "pct_alcohol",
    "Porcentaje de conductores involucrados en colisiones fatales que no estaban distraídos": "pct_no_distraidos",
    "Porcentaje de conductores involucrados en colisiones fatales que no habían estado involucrados en accidentes previos": "pct_sin_accidentes_previos",
    "Primas de seguro de automóvil ($)": "primas_seguro",
    "Pérdidas incurridas por las compañías de seguros por colisiones por conductor asegurado ($)": "perdidas_por_conductor",
}

# Aplicar el cambio
df = df.rename(columns=rename_dict)

# Verificar
df.head()

## 4. Tipos de datos, valores faltantes y duplicados

Antes de hacer modelos o gráficos, debemos asegurarnos de que:

- Cada columna tenga el **tipo de dato correcto** (numérico vs categórico).
- No haya **duplicados** problemáticos.
- Conozcamos la magnitud de los **valores faltantes**.


In [ ]:
# 4.1 Revisar tipos de datos
df.dtypes

In [ ]:
# Convertir `state` en tipo categórico (opcional pero recomendable)
df["estado"] = df["estado"].astype("category")

# Verificamos de nuevo
df.dtypes

In [ ]:
# 4.2 Valores faltantes (conteo y porcentaje)
conteo_nulos = df.isna().sum()
nulos_pct = (conteo_nulos / len(df) * 100).round(2)

resumen_nulos = pd.DataFrame({
    "conteo_nulost": conteo_nulos,
    "nulos_pct": nulos_pct
}).sort_values(by="nulos_pct", ascending=False)

resumen_nulos

In [ ]:
# 4.3 Filas duplicadas
num_duplicates = df.duplicated().sum()
print(f"Número de filas duplicadas en el dataset: {num_duplicates}")

### EJERCICIO 1 (para que programen ustedes)

1. Si existieran filas duplicadas, decidan si corresponde eliminarlas o no.  
2. Si existieran valores faltantes, definan una estrategia simple (por ejemplo, eliminar filas con NA en variables críticas).

> **Sugerencia**: usen `df.drop_duplicates()` y/o `df.dropna()` o imputación sencilla (media/mediana) si tiene sentido.


In [ ]:
# EJERCICIO 1: completa la limpieza según las decisiones de tu grupo.
# A modo de ejemplo, aquí dejamos líneas comentadas:

# 1) Eliminar filas completamente duplicadas
df = df.drop_duplicates()

# 2) Si quisieran eliminar filas con NA en alguna columna específica:
# cols_criticas = ["drivers_fatal_per_billion_miles", "pct_speeding", "pct_alcohol"]
# df = df.dropna(subset=cols_criticas)

# Muestra la nueva forma del dataframe
df.shape

## 5. Detección simple de outliers (z-score)

Ahora vamos a detectar **outliers** numéricos usando un criterio sencillo:

- Para cada columna numérica, calculamos el **z-score**.
- Marcamos como outlier aquellos registros con |z| > 3 en al menos una columna.

Esto no es perfecto, pero es un buen primer filtro.

In [ ]:
from scipy import stats

# Seleccionar solo columnas numéricas
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols

In [ ]:
# Calcular z-scores para cada columna numérica (ignorando filas con NA)
z_scores = np.abs(stats.zscore(df[numeric_cols], nan_policy='omit'))

# z_scores es un array; construimos una máscara de outliers
outlier_mask = (z_scores > 3).any(axis=1)

print(f"Número de filas marcadas como outlier (|z|>3 en alguna variable numérica): {outlier_mask.sum()}")

# Mostrar algunas filas outlier para inspección
df[outlier_mask].head()

In [ ]:
# Estadísticas descriptivas de las columnas numéricas
df.describe().T

## Outliers en la variable `pct_no_distraidos`

Al analizar la distribución de esta variable, se identificaron dos valores que se alejan significativamente del rango típico:

- **Mississippi** → `pct_no_distraidos = 10`
- **Wisconsin** → `pct_no_distraidos = 39`

Estos valores son **outliers** porque están muy por debajo del resto de los estados y pueden afectar análisis estadísticos y modelos predictivos.


In [ ]:
# EJERCICIO 2: decide qué hacer con los outliers.

# Ejemplo: eliminar outliers (si el grupo decide hacerlo)
# df_clean = df[~outlier_mask].copy()

# Ejemplo alternativo: mantenerlos, pero copiamos de todos modos
df_clean = df.copy()

df_clean.shape

## 6. Creación de variables nuevas

Vamos a crear algunas variables derivadas que pueden ser útiles luego para modelos:

1. **ratio_losses_premiums**: qué proporción de las primas de seguro se “pierde” en siniestros.  
   \[
   \text{ratio\_losses\_premiums} = \frac{\text{losses\_per\_insured\_driver}}{\text{car\_insurance\_premiums}}
   \]

2. **risk_speed_alcohol**: indicador simple de riesgo combinando % de choques con exceso de velocidad y % con alcohol.  
   \[
   \text{risk\_speed\_alcohol} = \text{pct\_speeding} + \text{pct\_alcohol}
   \]

3. **pct_safe_behaviour**: porcentaje aproximado de conductores que en choques fatales *no estaban distraídos* y *no tenían accidentes previos*.  
   (Esto es una aproximación solo para fines didácticos.)


In [ ]:
# 6.1 Crear ratio de pérdidas sobre primas
df_clean["ratio_losses_premiums"] = (
    df_clean["perdidas_por_conductor"] / df_clean["primas_seguro"]
)

# 6.2 Crear indicador combinado de riesgo (velocidad + alcohol)
df_clean["risk_speed_alcohol"] = (
    df_clean["pct_exceso_velocidad"] + df_clean["pct_alcohol"]
)

# 6.3 Aproximación de "comportamiento seguro" combinando no distraídos y sin accidentes previos
# Ojo: esto no es una fórmula oficial, solo una aproximación pedagógica
df_clean["pct_safe_behaviour"] = (
    df_clean["pct_no_distraidos"] * df_clean["pct_sin_accidentes_previos"] / 100.0
)

df_clean[[
    "estado",
    "primas_seguro",
    "perdidas_por_conductor",
    "ratio_losses_premiums",
    "risk_speed_alcohol",
    "pct_safe_behaviour"
]].head()


## 7. Selección de variables (X) y variable objetivo (y)

Para construir el modelo, definimos primero la **variable objetivo**, es decir, la columna que queremos predecir.  
En este caso, el objetivo es:

- **y:** `primas_seguro`

Luego seleccionamos las **variables predictoras (X)**, que representan factores asociados al riesgo vial y a las pérdidas históricas de las aseguradoras.  
Estas serán las entradas del modelo:

- `conductores_fatales_mm_millas`
- `pct_exceso_velocidad`
- `pct_alcohol`
- `pct_no_distraidos`
- `pct_sin_accidentes_previos`
- `perdidas_por_conductor`

Con estas columnas se construye el DataFrame final para el modelado:





In [ ]:
# ==========================
# 3. Seleccionar X e y 
# ==========================

# Variable objetivo (lo que queremos predecir)
target_col = "primas_seguro"

# Variables predictoras (características del modelo)
feature_cols = [
    "conductores_fatales_mm_millas",
    "pct_exceso_velocidad",
    "pct_alcohol",
    "pct_no_distraidos",
    "pct_sin_accidentes_previos",
    "perdidas_por_conductor"
]

# Creamos un DataFrame con solo las columnas necesarias
df_model = df_clean[feature_cols + [target_col]].copy()

df_model.head()



## 8. Checkpoint final del Notebook 1

Al terminar este notebook deberías tener:

- Un dataset con nombres de columnas limpios (`snake_case`).
- Tipos de datos adecuados (estado como categoría, resto numérico).
- Revisión (y decisión) sobre duplicados, valores faltantes y outliers.
- Nuevas variables derivadas con sentido de negocio.
- Un dataframe `model_df` listo para usar en el **Notebook 2 – Regresión Lineal**.

### Preguntas de reflexión (para 5 minutos finales)

1. ¿Qué variable crees que tendrá mayor impacto en las pérdidas de las aseguradoras (`losses_per_insured_driver`) y por qué?  
2. ¿Qué decisiones de limpieza podrían cambiar fuertemente los resultados de un futuro modelo?  
3. Si tuvieras que explicarle a alguien de negocio lo que hiciste en este notebook, ¿cómo lo resumirías en 3 frases?

En la próxima sesión vamos a:

- Dividir `model_df` en conjuntos de entrenamiento y prueba.
- Entrenar un modelo de **regresión lineal**.
- Evaluar métricas (MAE, RMSE, R²).
- Analizar la importancia de las variables.

Guarda este notebook: lo vas a reutilizar directamente.
